# Model Selection 모듈


- 훈련/테스트 데이터로 분리하지 않고 머신러닝 수행

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [2]:
iris = load_iris()
dtc = DecisionTreeClassifier(random_state=2021)
dtc.fit(iris.data, iris.target)
dtc.score(iris.data, iris.target)
# 암기 학습의 결과 : 정확도 100% 나옴 -> train, test로 나눔

1.0

- cross validate(): 교차 검증, validation하는 cv의 값은 5가 디폴트

In [7]:
from sklearn.model_selection import cross_validate
dtc = DecisionTreeClassifier()
res = cross_validate(dtc, iris.data, iris.target)
res
# 우리는 학습시간이 궁금한 것이 아니라, test_score가 궁금하다

{'fit_time': array([0.00099707, 0.00099635, 0.        , 0.        , 0.        ]),
 'score_time': array([0.        , 0.        , 0.00099683, 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])}

In [8]:
res['test_score'] # 아래에 있는 cross_val_score와 같다.

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

- cross_val_score()

In [6]:
from sklearn.model_selection import cross_val_score
cross_val_score(dtc, iris.data, iris.target, scoring='accuracy', cv=5)


array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

- GridSearchCV 클래스 : Hyper Parameter 튜닝 + 교차 검증
    - hyper parameter
        - 참고
            - https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-13-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0Parameter%EC%99%80-%ED%95%98%EC%9D%B4%ED%8D%BC-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0Hyper-parameter


In [9]:
# Train / Test dataset 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, stratify=iris.target, test_size=0.2, random_state=2021
)



In [16]:
# 분류기와 그에 해당하는 hyper parameter
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2021)
dtc.get_params()

# 'max_depth': None,'min_samples_split': 2가 비교적 많이 사용하는 parameter

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2021,
 'splitter': 'best'}

In [12]:
params ={
    'max_depth': [2,3,4,5,6],
    'min_samples_split': [2,3,4]
}

In [14]:
from sklearn.model_selection import GridSearchCV

grid_dt = GridSearchCV(
    dtc, param_grid=params, scoring='accuracy', cv=5
)

In [15]:
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [18]:
# 최적의 parameter
grid_dt.best_params_

{'max_depth': 4, 'min_samples_split': 2}

In [22]:
# 최적의 parameter로 학습한 분류기
best_clf = grid_dt.best_extimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_extimator_'

In [23]:
# 최적의 parameter로 학습한 분류기로 예측 및 평가
best_clf.score(X_test, y_test)

NameError: name 'best_clf' is not defined

- Support vector Machine

In [24]:
from sklearn.svm import SVC
svc = SVC(random_state=2021)
svc.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 2021,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [25]:
params = {'C': [0.01, 0.1, 1, 10, 100]}

In [26]:
grid_sv = GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5) # 최적의 C값을 찾아나가기 위한 과정 GridSearchCV
grid_sv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=2021),
             param_grid={'C': [0.01, 0.1, 1, 10, 100]}, scoring='accuracy')

In [28]:
grid_sv.best_params_
# best_params_이 최선의 결과인지 아직은 알 수 없다.

{'C': 10}

In [29]:
params = {'C':[3, 6, 10, 15, 20]} # C의 범위를 좁혀나가는 과정
grid_sv =  GridSearchCV(svc, param_grid=params, scoring='accuracy', cv=5)
grid_sv.fit(X_train, y_train)
grid_sv.best_params_

{'C': 3}

In [31]:
best_svc = grid_sv.best_estimator_
best_svc.score(X_test, y_test)

0.9666666666666667